In [ ]:
#!/opt/conda/envs/lae2020/bin/python3

from importlib.abc import Loader
import sys
import os
import ruamel.yaml

sys.path.insert(0,"/home/chinahg/GCresearch/cantera/build/python")
sys.path.insert(1,"/home/chinahg/GCresearch/rocketemissions")

import cantera as ct
ct.add_directory('/user/chinahg')
ct.__file__

import h5py
import numpy as np
import time
import math as math
import matplotlib.pyplot as plt
import scipy as sp
import scipy.optimize
import yaml
import shutil
from scipy import integrate
import matplotlib.pyplot as plt

In [ ]:
# n_species = 32
# upper = 20000 #[m]
# lower = 16000 #[m]
# space = int((upper - lower) / 2000 + 1) #250
# h = [16000] #Int.(LinRange(16000, 20000, space))
# g = 0

# for m in range(len(h)):
#     h_string = str(h[m])

#     ### IMPORT SHOCK EXIT CONDITIONS ###
#     hdf = h5py.File('plot_data.h5', 'r')
#     with hdf:
#         hdf = h5py.File('plot_data.h5', 'r')

#         p = hdf[h_string + "m/P"][2]
    
#         #curve fit #a = ambient vel [m/s] (speed of rocket) 
#         u_a = 1.276E-11*int(h[m]^3) - 9.465E-07*(h[m]^2) + 4.308E-02*h[m] + 1.257E+02
        
#         X0_full = hdf[h_string + "m/X"][()]
#         X0 = np.zeros(n_species)
        
#         #Redefine species initial conditions for multiple species:
#         for d in range(len(X0_full[1,:])-1):
#             X0[d] = X0_full[1,d]

#         X_a = np.zeros(n_species)
#         X_a[9] = 0.780790 #ppm N2
#         X_a[3] = 0.209445 #ppm O2
#         X_a[8] = 0.9339+0.426 #ppm Ar
        
#         T_a = hdf[h_string + "m/T_a"][()]
        
#         T0 = hdf[h_string + "m/T"][2] #initial plume temperature
    
#         u0 = hdf[h_string + "m/u"][1] #initial plume velocity
# print(u0)
# print(p)
# print(T0)
# print(T_a)
# print(u_a)

In [ ]:
mdot_air = 0.16*u_a # [kg/s] Entrained air
r0 = 1.45
mdot_exhaust = 67.35+404.79 #exhaust mdot [kg/s]

n_species = 32
upper = 20000 #[m]
lower = 16000 #[m]
space = int((upper - lower) / 2000 + 1) #250
h = np.linspace(16000, 20000, space, dtype=int)
g = 0

for m in range(len(h)):
    h_string = str(h[m])

    ### IMPORT SHOCK EXIT CONDITIONS ###
    hdf = h5py.File('plot_data.h5', 'r')
    with hdf:
        hdf = h5py.File('plot_data.h5', 'r')

        p = hdf[h_string + "m/P"][2]
    
        #curve fit #a = ambient vel [m/s] (speed of rocket) 
        u_a = 1.276E-11*int(h[m]^3) - 9.465E-07*(h[m]^2) + 4.308E-02*h[m] + 1.257E+02
        
        X0_full = hdf[h_string + "m/X"][()]
        X0 = np.zeros(n_species)
        
        #Redefine species initial conditions for multiple species:
        for d in range(len(X0_full[1,:])-1):
            X0[d] = X0_full[1,d]

        X_a = np.zeros(n_species)
        X_a[9] = 0.780790 #ppm N2
        X_a[3] = 0.209445 #ppm O2
        X_a[8] = 0.9339+0.426 #ppm Ar
        
        T_a = hdf[h_string + "m/T_a"][()]
        
        T0 = hdf[h_string + "m/T"][2] #initial plume temperature
    
        u0 = hdf[h_string + "m/u"][1] #initial plume velocity

    #WSR
    #Create Gas Mixture
    gasFuel = ct.Solution('A2NOx.yaml') #ct.Hydrogen()
    gasFuel.TPX = T0, p, X0 #https://science.ksc.nasa.gov/shuttle/technology/sts-newsref/et.html#:~:text=LIQUID%20HYDROGEN%20TANK,-The%20liquid%20hydrogen&text=Its%20operating%20pressure%20range%20is,to%20the%20left%20aft%20umbilical.
    gasOx = ct.Solution('A2NOx.yaml') #ct.Oxygen()
    gasOx.TPX = T_a, p, X_a
    gasFuel.equilibrate('HP')
    
    #Create Reactor Infrastructure
    #tankFuel = ct.Reservoir(gasFuel)
    tankOx = ct.Reservoir(gasOx)
    CC_reactor = ct.IdealGasConstPressureReactor(gasFuel) #create CC with random gas initially filling it
    CC_reactor.volume = np.pi*r0**2*u0
    
    
    #Connect tanks to CC
    #flow_controller_fuel = ct.MassFlowController(upstream=tankFuel,downstream=CC_reactor,mdot=mdot_exhaust)
    flow_controller_ox = ct.MassFlowController(upstream=tankOx,downstream=CC_reactor,mdot=mdot_air)
    
    #Connect CC to exhaust reservoir
    CC_reactorNet = ct.ReactorNet([CC_reactor])
    
    #React!
    t = 0
    state = ct.SolutionArray(gasOx, extra=['t','volume','eox'])
    while t < (15):
        CC_reactorNet.step()
        t = CC_reactorNet.time
    
        # Extract the state of the reactor
        state.append(CC_reactor.thermo.state, volume=CC_reactor.volume, t=t, eox=gasOx.enthalpy_mole)
    # Plot results
    f, ax1 = plt.subplots(1, 1)
    ax1.plot(state.t*r0, state.T, color='C3')
    ax2 = ax1.twinx()
    ax2.plot(state.t*r0, state.volume, color='C0')
    ax1.set_xlabel('x/r0')
    ax2.set_ylabel('Volume [m^3]', color='C0')
    ax1.set_ylabel('Temperature [K]', color='C3')
    ax1.set_title("Altitude: "+h_string+" m")
    
    f.tight_layout()
    plt.show()
    
    EI_NO = state.Y[:,11]*(state.volume*state.density_mass)*10000/(state.volume[0]*state.density_mass[0])
    
    f, ax1 = plt.subplots(1, 1)
    ax1.plot(state.t*r0, state.T, color='C3')
    ax2 = ax1.twinx()
    ax2.plot(state.t*r0, EI_NO, color='C6')
    ax1.set_xlabel('x/r0')
    ax2.set_ylabel('EI NO [g NO/kg exhaust]', color='C6')
    ax1.set_ylabel('Temperature [K]', color='C3')
    
    
    f.tight_layout()
    plt.show()
    f,ax1 = plt.subplots()
    
    legend = ['H2', 'H','O', 'O2', 'OH', 'H2O',  'H2O2', 'HO2', 'AR', 'N2', 
        'OH_star', 'NO', 'NO2', 'NH3', 'NH2', 'NH', 'N', 'NNH', 'N2H4', 'N2H3', 'N2H2', 'H2NN', 
        'NH2OH', 'H2NO', 'HNOH', 'HNO', 'HON', 'HONO', 'HNO2', 'NO3', 'HONO2', 'N2O']
    
    
    for i in range(n_species-1):
        if np.all(state.X[:,i] != 0):
            ax1.plot(state.t/r0,state.X[:,i], label=legend[i])
    
    # Shrink current axis by 20%
    box = ax1.get_position()
    ax1.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    # Put a legend to the right of the current axis
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# #######################################################################
# # Input Parameters
# #######################################################################
# mdot_air = 0.16*u0 # [kg/s] Entrained air
# r0 = 1.45
# mdot_exhaust = 67.35+404.79 #exhaust mdot [kg/s]

# #Create Gas Mixture
# gasExhaust = ct.Solution('A2NOx.yaml')
# gasExhaust.TPY = T0, p, X0

# gasOx = ct.Solution('A2NOx.yaml') #ct.Oxygen()
# gasOx.TPX = T_a, p, X_a

# mass_exhaust = np.sum(mdot_exhaust*1*gasExhaust.Y)
# mass_air = np.sum(mdot_air*1*gasOx.Y)
# Y_exhaust = mdot_exhaust*1*gasExhaust.Y
# Y_air = mdot_air*1*gasOx.Y
# mass_tot_i = Y_exhaust+Y_air
# mass_fractions_i = mass_tot_i/(mass_air+mass_exhaust)

# T_0 = 1845  # inlet temperature [K]
# pressure = ct.one_atm  # constant pressure [Pa]
# composition_0 = mass_fractions_i # time determined by velocity (equal to 200m)
# length = 200  # *approximate* PFR length [m]
# area = 1.45  # cross-sectional area [m**2]

# # input file containing the reaction mechanism
# reaction_mechanism = '/home/chinahg/GCresearch/rocketemissions/A2NOx.yaml'

# # Resolution: The PFR will be simulated by 'n_steps' time steps or by a chain
# # of 'n_steps' stirred reactors.
# n_steps = 2000
# #####################################################################


# #####################################################################
# # Method 1: Lagrangian Particle Simulation
# #####################################################################
# # A Lagrangian particle is considered which travels through the PFR. Its
# # state change is computed by upwind time stepping. The PFR result is produced
# # by transforming the temporal resolution into spatial locations.
# # The spatial discretization is therefore not provided a priori but is instead
# # a result of the transformation.

# # import the gas model and set the initial conditions
# gas1 = ct.Solution(reaction_mechanism)
# gas1.TPX = T_0, pressure, composition_0
# mass_flow_rate1 = u0 * gas1.density * area

# # create a new reactor
# r1 = ct.IdealGasConstPressureReactor(gas1)
# # create a reactor network for performing time integration
# sim1 = ct.ReactorNet([r1])

# # approximate a time step to achieve a similar resolution as in the next method
# t_total = length / u0
# dt = t_total / n_steps
# # define time, space, and other information vectors
# t1 = (np.arange(n_steps) + 1) * dt
# z1 = np.zeros_like(t1)
# u1 = np.zeros_like(t1)
# states1 = ct.SolutionArray(r1.thermo)
# for n1, t_i in enumerate(t1):
#     # perform time integration
#     sim1.advance(t_i)
#     # compute velocity and transform into space
#     u1[n1] = mass_flow_rate1 / area / r1.thermo.density
#     z1[n1] = z1[n1 - 1] + u1[n1] * dt
#     states1.append(r1.thermo.state)

# # Plot results
# f, ax1 = plt.subplots(1, 1)
# ax1.plot(z1, states1.T, color='C1')
# ax1.set_xlabel('x/r0')
# ax1.set_ylabel('temperature [K]', color='C1')
# plt.show()

In [ ]:
# #Create Gas Mixture

# mdot_ox = 404
# mdot_f = 67.4
# r0 = 1.45
# gasExhaust = ct.Solution('h2o2.yaml')
# gasExhaust.TPY = 300, p, 'O2: 6, H2:1'
# gasExhaust.equilibrate('HP')

# gasFuel = ct.Solution('h2o2.yaml') #ct.Hydrogen()
# gasFuel.TPX = 20.15, 227527, 'H2:1' #https://science.ksc.nasa.gov/shuttle/technology/sts-newsref/et.html#:~:text=LIQUID%20HYDROGEN%20TANK,-The%20liquid%20hydrogen&text=Its%20operating%20pressure%20range%20is,to%20the%20left%20aft%20umbilical.
# gasOx = ct.Solution('h2o2.yaml') #ct.Oxygen()
# gasOx.TPX = 90.15, 140000, 'O2:1'

# #Fuel and ox through low pressure turbopump and heat exchanger
# gasFuel.TP = 533.15, p #1000000
# gasOx.TP = 145.9, 2900000 #weighted average of inlet ox temps

# #Create Reactor Infrastructure
# tankFuel = ct.Reservoir(gasFuel)
# tankOx = ct.Reservoir(gasOx)
# CC_exhaust1 = ct.Reservoir(gasExhaust)

# CC_reactor = ct.IdealGasReactor(gasExhaust) #create CC with random gas initially filling it

# #Connect tanks to CC
# flow_controller_fuel = ct.MassFlowController(upstream=tankFuel,downstream=CC_reactor,mdot=mdot_f)
# flow_controller_ox = ct.MassFlowController(upstream=tankOx,downstream=CC_reactor,mdot=mdot_ox)

# #flow_controller_fuel2 = ct.MassFlowController(upstream=CC_reactor,downstream=CC_exhaust1,mdot=mdot_f+mdot_ox)
# #flow_controller_ox2 = ct.MassFlowController(upstream=CC_reactor,downstream=CC_exhaust1,mdot=mdot_ox)
# #Connect CC to exhaust reservoir
# press_controller_ox = ct.PressureController(upstream=CC_reactor, downstream=CC_exhaust1, master=flow_controller_ox, K=0.001)
# press_controller_fuel = ct.PressureController(CC_reactor, CC_exhaust1, master=flow_controller_fuel, K=0.001)
# CC_reactorNet = ct.ReactorNet([CC_reactor])
# #React!
# t = 0
# state = ct.SolutionArray(gasOx, extra=['t','volume','enthalpy','eox','efuel'])
# while t < 0.2:
#     CC_reactorNet.step()
#     t = CC_reactorNet.time
#     #CC_reactorNet.advance(t)
    
#     # Extract the state of the reactor
#     state.append(CC_reactor.thermo.state, volume=CC_reactor.volume, t=t, enthalpy=gasExhaust.enthalpy_mole, efuel=gasFuel.enthalpy_mole, eox=gasOx.enthalpy_mole)
# print(state.volume_mass)
# # Plot results
# f, ax1 = plt.subplots(1, 1)
# ax1.plot(state.t, state.T, color='C1')
# ax1.set_xlabel('x/r0')
# ax1.set_ylabel('temperature [K]', color='C1')

# plt.show()